In [67]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

In [68]:
dc_evt = pd.read_csv('data/dc_evt_modified2.csv', low_memory=False)
dc_file = pd.read_csv('data/dc_file_modified2.csv', low_memory=False)

In [69]:
binary_col_file = [col for col in dc_file.columns if set(dc_file[col])|{0, 1} == {0, 1}]
binary_col_evt = [col for col in dc_evt.columns if set(dc_evt[col])|{0, 1} == {0, 1}]

In [70]:
bin_file_ag = dc_file[['inode', 'filename']+binary_col_file]
bin_file_ag['finode'] = bin_file_ag['inode'].astype(str) +"-"+ bin_file_ag["filename"]
bin_file_ag = bin_file_ag.drop(['inode'], axis=1)
bin_file_ag = bin_file_ag.drop(['filename'], axis=1)
bin_file_ag = bin_file_ag.groupby('finode').sum()

bin_evt_ag = dc_evt[['inode']+binary_col_evt].groupby('inode').sum()

/tmp/ipykernel_29249/2879601168.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bin_file_ag['finode'] = bin_file_ag['inode'].astype(str) +"-"+ bin_file_ag["filename"]


In [76]:
model = IsolationForest(n_estimators=200, contamination='auto', n_jobs=-1, max_samples=len(bin_file_ag))
preds = model.fit_predict(bin_file_ag)
len(np.where(preds == -1)[0])

1020